# Connect and Insert Data to MySQL Server with Python

# Introduction

In the world with increasing data availability, it is become a norm to store and collect data from a cloud database instead of using a local file when you are part of a start up or large corporation. Several service, such as Google Cloud Platform or Microsoft Azure provide us with a cloud storage service to store a large dataset. Most of the current database system use SQL to store and collect this data. Therefore, understanding how to write a command or a query using SQL is currently one of the top skills required in data-related job, espescially for a data analyst and data scientist. The following graph is the result of [2020 Data Science and Machine Learning Survey](https://research.aimultiple.com/data-science-tools/).

<center><img src="asset/survey.png"></center>

To learn more about how to run an SQL query, we will use one of the most common database management system: MySQL. On this occasion, I will guide you on how to do the following things with MySQL:

- Create a database with MySQL server
- Create multiple table
- Insert data into SQL table 
- Write query to collect data from SQL database

The full data analytics process that will analyze and gain insight from the data will be done in separate notebook.

For a quick introduction about SQL if you are not familiar with SQL, you can visit [this website](https://www.learnsqlonline.org/) and just read the welcome page or you can try some practice and come back here later.

# Set Up MySQL Server

There are a lot of options for you to start creating a database, either using common cloud service such as [Google Cloud Platform](https://help.appsheet.com/en/articles/3627169-create-a-mysql-database-hosted-in-google-cloud) or [Microsoft Azure](https://docs.microsoft.com/en-us/azure/mysql/flexible-server/quickstart-create-server-portal), or you can also try setting up a local mysql server in your device. However, setting up a database on these can be a quite long process. Since our goal is focus on preparing data and store in in mysql, we will use a free hosting website with [db4free](https://db4free.net/). 

![](asset/db4free.png)

This website help us set a free and small MySQL server for us to practice. You just need to [register](https://db4free.net/signup.php) with your email and enter the following information:

- The name of your MySQL database
- Your username to login
- Your password to login
- Your email address for validation

# Library

The following is the required library that we will use throughout this notebook.

In [1]:
# Data wrangling
import pandas as pd
import numpy as np

# Regular expression to transform string
import re

# Computation time
import time

# Connect to mysql server
import mysql.connector

# Basic SQL

## Connect to MySQL Server

After you have set up the mysql server, now you can connect into the server with RMySQL. You need the following information from the server:

- host name
- port
- username
- password
- database name (dbname)

In [2]:
mydb = mysql.connector.connect(
    host = "db4free.net",
    port = 3306,
    user = "your_name",
    password = "your_password",
    database = "your_dbname"
)

print(mydb)

## Create and Check Table

Let's check the connection and start with the basic. We will create a simple table of customer information with 3 columns:

- `customer_id`: ID of the customer with data type of integer (INT)
- `name`: name of the customer with data type of character (VARCHAR 100)
- `city`: city location of the customer with data type of character (VARCHAR 100)

____________

**Trivia**

VARCHAR(100) means that we state that this column has data type of character with maximum length of 100 characters.

____________

The following is the general SQL command for creating a table

> CREATE TABLE table_name(
column_1 type_data1,
column_2 type_data2,
)



In [3]:
query = """
CREATE TABLE dummy_customer(
customer_id INT,
name VARCHAR(100),
city VARCHAR(100)
)
"""

query

'\nCREATE TABLE dummy_customer(\ncustomer_id INT,\nname VARCHAR(100),\ncity VARCHAR(100)\n)\n'

After you have created the SQL query, you can execute them by creating a cursor from your connection (`mydb`) object.

In [4]:
# Create cursor 
cursor = mydb.cursor() 

# Run query 
cursor.execute(query) 

# Commit for creating table or manipulating table structure 
mydb.commit() 

# Close cursor
cursor.close()

True

You can use `read_sql()` function to check the description of your table from MySQL server using the following command. 

> DESCRIBE table_name

Some information about the output:

- Field: name of the column
- Type: data type
- Null: is missing value allowed?
- Key: is this column a primary key (PRI) or a foreign key?

In [5]:
pd.read_sql("DESCRIBE dummy_customer", mydb)

,Field,Type,Null,Key,Default,Extra
0,customer_id,b'int',YES,,None,
1,name,b'varchar(100)',YES,,None,
2,city,b'varchar(100)',YES,,None,


You can check all available table in the database by using the following query. This will return all available table in your database in a dataframe format.

> SHOW TABLES

In [6]:
pd.read_sql("SHOW TABLES", mydb)

,Tables_in_db_zeed
0,dummy_customer
1,host_info
2,listing


If you want to delete the table, you can run the following query.

In [7]:
# Create cursor 
cursor = mydb.cursor() 

# Run query 
cursor.execute("DROP TABLE dummy_customer") 

# Close cursor
cursor.close()

True

# Data

We will use data of Thailand room listing from [Airbnb](http://insideairbnb.com/get-the-data.html). The dataset contain information about room or listing that is posted on Airbnb site in Bangkok, Central Thailand, Thailand.

We will use the `Listing` dataset that contain the detailed information about listing or room posted on airbnb. One host can have multiple listings. For the detailed information regarding each column, you can check the description for each column in [this spreadsheet](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit?usp=sharing).

The dataset can be further broken down into several parts. For example, in listing data there are several columns that store information about the host instead of the room/listing. In practice, we can reduce the number of required space for the database by separating the data about listing and the data about host, since a single host can have multiple listing. 

To illustrate the relation between each data, we will draw an **Entity Relationship Diagram (ERD)**. An ERD is often used to help data engineer to design a database and show the relation between each table. 

<center><img src="asset/ERD_Airbnb.png"/></center>

On each table, you can have a column that contain a unique ID to identify each row in a table. This column is called as **Primary Key (PK)**. For example, in host_info table, a unique host should only stored once and there is no duplicate in the table. This host is identified by the `host_id` column. There is also a column that is called as **Foreign Key (FK)**. For example, the listing table has `id` as the primary key and `host_id` as the foreign key. This means that we can join the listing table with the host_info table by matching the `host_id` on the listing table with the `host_id` on the host_info table. 

The relation or the cardinality between table is illustrated by the sign at both end of the arrow. There are different cardinality in ERD, you can look at the detailed explanation [here](https://www.youtube.com/watch?v=QpdhBUYk7Kk). A host can have zero (no listing) or many listing, so the cardinality at the listing side is illustrated as *zero or many*. Naturally, a room listing can only be owned by a single host. There is no listing that is owned by different hosts. Therefore, the cardinality at the host_info side is illustrated as *one (and only one)*.

<center><img src="asset/erd_cardinality.jpg" style="width: 400px;"/></center>

## Host

### Processing Host Table

Since the only table that has no foreign key is `host_info`, we will start creating and cleansing a host table from the listing data. Let's start by importing the listing data.

In [8]:
df_listing = pd.read_csv("data/listings.csv")

print("Number of rows: {:,}".format(df_listing.shape[0]))
print("Number of columns: {:,}".format(df_listing.shape[1]))

Number of rows: 19,289
Number of columns: 74


Let's check the some of the content from the dataset.

In [9]:
df_listing.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,27934,https://www.airbnb.com/rooms/27934,20210321015140,2021-03-22,Nice room with superb city view,Our cool and comfortable one bedroom apartment...,It is very center of Bangkok and easy access t...,https://a0.muscache.com/pictures/566374/23157b...,120437,https://www.airbnb.com/users/show/120437,...,10.0,9.0,9.0,NaN,f,2,2,0,0,0.6
1,27942,https://www.airbnb.com/rooms/27942,20210321015140,2021-03-22,Town House (1) bedroom available,<b>The space</b><br />Townhouse with 2 bedroom...,NaN,https://a0.muscache.com/pictures/155668/afc77b...,120462,https://www.airbnb.com/users/show/120462,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
2,27979,https://www.airbnb.com/rooms/27979,20210321015140,2021-03-31,"Easy going landlord,easy place","<b>The space</b><br />""The village Condominium...",NaN,https://a0.muscache.com/pictures/106247594/1d6...,120541,https://www.airbnb.com/users/show/120541,...,NaN,NaN,NaN,NaN,f,2,1,1,0,NaN
3,28354,https://www.airbnb.com/rooms/28354,20210321015140,2021-04-01,Place near Metro and Shopping Mall ( Monthly o...,Apartment is near 2 shopping mall Central Plaz...,NaN,https://a0.muscache.com/pictures/3097966/efa67...,121848,https://www.airbnb.com/users/show/121848,...,10.0,9.0,9.0,NaN,t,1,1,0,0,0.4
4,28745,https://www.airbnb.com/rooms/28745,20210321015140,2021-03-30,modern-style apartment in Bangkok,A modern-style apartment situated in a cool ne...,Ramkumheang neigbourhood - hip place with food...,https://a0.muscache.com/pictures/160369/92a7ec...,123784,https://www.airbnb.com/users/show/123784,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN


Let's check the information from the dataset.

In [10]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19289 entries, 0 to 19288
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            19289 non-null  int64  
 1   listing_url                                   19289 non-null  object 
 2   scrape_id                                     19289 non-null  int64  
 3   last_scraped                                  19289 non-null  object 
 4   name                                          19280 non-null  object 
 5   description                                   18118 non-null  object 
 6   neighborhood_overview                         11099 non-null  object 
 7   picture_url                                   19289 non-null  object 
 8   host_id                                       19289 non-null  int64  
 9   host_url                                      19289 non-null 

The listing dataset contain information about each room listing or room posted on the airbnb website by the host.

A single host is identified by the `host_id` and may have multiple room in the listing. Let's check this hypothesis.

In [11]:
df_listing.value_counts('host_id')

host_id
201677068    178
39669202     121
187786849     82
19040245      77
8663142       72
            ... 
71745725       1
71650969       1
71579199       1
71577576       1
392928219      1
Length: 8296, dtype: int64

As we have seen from the above output, several hosts even have hundreds of listing. For efficient storage in the database, we will separate information about the host, such as `host_id`, `host_url`, etc. from the information about the listing.

In [12]:
# Collect columns related to host 
df_host1 = df_listing.loc[:, 'host_id':'host_identity_verified']
df_host2 = df_listing.loc[:, 'calculated_host_listings_count':'calculated_host_listings_count_shared_rooms']

df_host = pd.concat([df_host1, df_host2], axis = 1)

Now we have 22 columns for the host and the rest is columns related to the individual listing.

There should be no duplicate host, so we will remove duplicated host.

In [13]:
df_host = df_host[ df_host.duplicated('host_id') == False ]

print("Number of rows: %i" %df_host.shape[0])

Number of rows: 8296


The next we do is preparing the dataset so they will have a proper data type in the database. You may have noticed that some columns should be have a boolean or logical data type, such as the `host_is_superhost`, `host_has_profile_pic`, and `host_identity_verified`. They contain string *t* if the value is *True* and *f* if the value is *False*. We will transform the data to the proper data type.

In [14]:
for col in ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified']:
    
    df_host[col] = list(map(lambda x: True if x == "t" else False if x == "f" else np.nan, 
                            df_host[col]
                           )
                       )

The next thing that I want to clean up is the `host_verification` column that is in a list format like following sample.

In [15]:
df_host['host_verifications'][0:5]

0    ['email', 'phone', 'reviews', 'jumio', 'offlin...
1                                   ['email', 'phone']
2                                   ['email', 'phone']
3    ['email', 'phone', 'facebook', 'reviews', 'jum...
4                                   ['email', 'phone']
Name: host_verifications, dtype: object

We will clean them so they contain a simple string text. For example, the first row will be *email, phone, facebook, ...*.

In [16]:
# Remove square bracket and apostrophe
df_host['host_verifications'] = list(map(lambda x: re.sub("[\'\[\]]", '', x), df_host['host_verifications']))

df_host['host_verifications'][0:5]

0    email, phone, reviews, jumio, offline_governme...
1                                         email, phone
2                                         email, phone
3    email, phone, facebook, reviews, jumio, selfie...
4                                         email, phone
Name: host_verifications, dtype: object

### Create Host Table

We will start creating a table for the database. Since there is no numeric or decimal value from the host data, we will assign all numeric value into integer. The next thing we need to do is to design a proper data type for the string text columns. We will check the maximum length of each string column in the host dataset.

In [17]:
# Function to check maximum character length
def check_char(data):
    print("Maximum Character Length")
    
    for col in data.columns:
        char_length = list(map(lambda x: len( str(x) ), data[col]))
        print(col + ": %i" %np.max(char_length))

In [18]:
check_char(df_host.select_dtypes('object'))

Maximum Character Length
host_url: 43
host_name: 35
host_since: 10
host_location: 125
host_about: 9060
host_response_time: 18
host_response_rate: 4
host_acceptance_rate: 4
host_is_superhost: 5
host_thumbnail_url: 106
host_picture_url: 109
host_neighbourhood: 28
host_verifications: 140
host_has_profile_pic: 5
host_identity_verified: 5


Now we can create a table named `host_info` with the following data type:

- INT: integer
- VARCHAR(n): String or character with maximum length of n
- DATE: date (format in YYYY-MM-DD)
- BOOLEAN: logical (TRUE or FALSE)

Make sure the number of character (n) in VARCHAR is bigger than the length of your data characters. For example, the maximum length of `host_location` is 125, so you can create a VARCHAR(200) or VARCHAR(500) just to make sure. Don't forget to assign `host_id` as the primary key for the table.

For more detailed information about different data type allowed by MySQL, you can check the following [manuals](https://dev.mysql.com/doc/refman/8.0/en/data-types.html).

In [19]:
query =  """
CREATE TABLE host_info(
host_id INT,
host_url VARCHAR(50),
host_name VARCHAR(100),
host_since DATE,
host_location VARCHAR(500),
host_about VARCHAR(10000),
host_response_time VARCHAR(50),
host_response_rate VARCHAR(50),
host_acceptance_rate VARCHAR(50),
host_is_superhost BOOLEAN,
host_thumbnail_url VARCHAR(500),
host_picture_url VARCHAR(500),
host_neighbourhood VARCHAR(50),
host_listings_count INT,
host_total_listings_count INT,
host_verifications VARCHAR(500),
host_has_profile_pic BOOLEAN,
host_identity_verified BOOLEAN,
calculated_host_listings_count INT,
calculated_host_listings_count_entire_homes INT,
calculated_host_listings_count_private_rooms INT,
calculated_host_listings_count_shared_rooms INT,
PRIMARY KEY(host_id)
)
"""

query

'\nCREATE TABLE host_info(\nhost_id INT,\nhost_url VARCHAR(50),\nhost_name VARCHAR(100),\nhost_since DATE,\nhost_location VARCHAR(500),\nhost_about VARCHAR(10000),\nhost_response_time VARCHAR(50),\nhost_response_rate VARCHAR(50),\nhost_acceptance_rate VARCHAR(50),\nhost_is_superhost BOOLEAN,\nhost_thumbnail_url VARCHAR(500),\nhost_picture_url VARCHAR(500),\nhost_neighbourhood VARCHAR(50),\nhost_listings_count INT,\nhost_total_listings_count INT,\nhost_verifications VARCHAR(500),\nhost_has_profile_pic BOOLEAN,\nhost_identity_verified BOOLEAN,\ncalculated_host_listings_count INT,\ncalculated_host_listings_count_entire_homes INT,\ncalculated_host_listings_count_private_rooms INT,\ncalculated_host_listings_count_shared_rooms INT,\nPRIMARY KEY(host_id)\n)\n'

After you have created the SQL query, you can execute them.

```
# Create cursor 
cursor = mydb.cursor() 

# Run query 
cursor.execute(query) 

# Commit for creating table or manipulating table structure 
mydb.commit() 

# Close cursor
cursor.close()
```

You can run a query and get a result. For example, you can check data information about the `host_info` table.

In [20]:
pd.read_sql("DESCRIBE host_info", mydb)

,Field,Type,Null,Key,Default,Extra
0,host_id,b'int',NO,PRI,None,
1,host_url,b'varchar(50)',YES,,None,
2,host_name,b'varchar(100)',YES,,None,
3,host_since,b'date',YES,,None,
4,host_location,b'varchar(500)',YES,,None,
5,host_about,b'varchar(10000)',YES,,None,
6,host_response_time,b'varchar(50)',YES,,None,
7,host_response_rate,b'varchar(50)',YES,,None,
8,host_acceptance_rate,b'varchar(50)',YES,,None,
9,host_is_superhost,b'tinyint(1)',YES,,None,


If you want to check the content or the data from `host_info` table, you can use the following query, although for now the table is still empty because we don't insert anything yet into the table.

```
pd.read_sql("SELECT * FROM host_info", mydb)
```

### Insert Data to Host Table

Let's start inserting data into MySQL server. The generic formula to insert data into the table is as follows:

> INSERT INTO table_name (column_name) VALUES (value_for_each_column)

For example, if you have a table named **customer** that has `reviewer_id` and `reviewer_name` column, you can insert a row with the following query:

> INSERT INTO customer (reviewer_id, reviewer_name) VALUES (123, 'John Doe')

So we need to insert the name of the column and the respective value for each column. Let's start by preparing the column name into a single string.

In [21]:
# Get column name
column_name = df_host.columns.to_list()

# Convert column name to single string 
column_name = ",".join(x for x in column_name)

column_name

'host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms'

Next, we can prepare the value into a single string for each row. For example, the following is the input for the first row of the host data.

Just like when you read a csv file, an input for a single column is separated by comma (,) value by default. For example, input for the first column `host_id` is 266763, input for the second column is the string for `host_url`, etc. However, you may notice that at the middle of the string we have an extra comma from the `host_verifications` (email, phone, facebook) which should go into a single column. We need to clean the string so they can fit properly to how the SQL will read the data.

To get a clean values, we need to process the string column first with the following rules:

- A string should be started and ended with quotation mark (""), e.g. "email, phone, facebook", "Fransesca"
- Missing values (NA) should not be quoted and should be transformed to NULL for SQL
- Logical value should not be quoted

To quickly clean our data, we will define a function that will help us.

In [22]:
def clean_char(data):
    
    # Replace None with numpy.nan
    data.fillna(value=np.nan, inplace = True)
    
    for col in data.select_dtypes('object').columns:
        
        # Get index with no missing values
        non_na = data[col][ data[col].isna() == False ].index
        
        # Change quotation mark with apostrophe
        data.loc[non_na, col] = list(map(lambda x: re.sub('"', "'", str(x)), data.loc[non_na, col] ))
        
        # Add quotation mark at the start and end of string
        data.loc[non_na, col] = list(map(lambda x: '"' + x + '"', data.loc[non_na, col]))
               
        # Remove quotation mark from logical value (True and False)
        data.loc[non_na, col] = list(map(lambda x: re.sub('"True"', 'True', x), data.loc[non_na, col]))
        data.loc[non_na, col] = list(map(lambda x: re.sub('"False"', 'False', x), data.loc[non_na, col]))
        
    return(data)

Let's apply the function to our host data.

In [23]:
df_host = clean_char(df_host)

df_host.head()

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,120437,"""https://www.airbnb.com/users/show/120437""","""Nuttee""","""2010-05-08""","""Bangkok""","""Hi All, I am nuttee patranavik from Bangkok, ...","""within a few hours""","""100%""",NaN,False,...,"""Victory Monument""",2.0,2.0,"""email, phone, reviews, jumio, offline_governm...",True,True,2,2,0,0
1,120462,"""https://www.airbnb.com/users/show/120462""","""Donald""","""2010-05-08""","""Thailand""",NaN,NaN,NaN,NaN,False,...,NaN,1.0,1.0,"""email, phone""",True,False,1,0,1,0
2,120541,"""https://www.airbnb.com/users/show/120541""","""Emy""","""2010-05-08""","""Bangkok, Thailand""",NaN,NaN,NaN,NaN,False,...,NaN,2.0,2.0,"""email, phone""",True,False,2,1,1,0
3,121848,"""https://www.airbnb.com/users/show/121848""","""Jaruwan""","""2010-05-10""","""Bangkok, Thailand""","""Hi I'm Jaruwan. I love to prepare everything ...","""within a day""","""100%""",NaN,False,...,"""Chatuchak""",1.0,1.0,"""email, phone, facebook, reviews, jumio, selfi...",True,True,1,1,0,0
4,123784,"""https://www.airbnb.com/users/show/123784""","""Familyroom""","""2010-05-12""","""Bangkok Thailand""","""Welcome to Familyroom Apartment.\r\n\r\nA Chi...",NaN,NaN,NaN,False,...,"""Bang Kapi""",1.0,1.0,"""email, phone""",True,False,1,0,1,0


Let's check the input value for the second observation.

In [24]:
# Convert value of each row into a single string
value_data = ", ".join(str(x) for x in df_host.iloc[0])

value_data

'120437, "https://www.airbnb.com/users/show/120437", "Nuttee", "2010-05-08", "Bangkok", "Hi All, I am nuttee patranavik from Bangkok, Thailand.\r\nalways travel but easy to connect via airbnb..", "within a few hours", "100%", nan, False, "https://a0.muscache.com/im/pictures/user/d05a6b51-b459-4eb8-a01e-8c55122b0132.jpg?aki_policy=profile_small", "https://a0.muscache.com/im/pictures/user/d05a6b51-b459-4eb8-a01e-8c55122b0132.jpg?aki_policy=profile_x_medium", "Victory Monument", 2.0, 2.0, "email, phone, reviews, jumio, offline_government_id, selfie, government_id, identity_manual", True, True, 2, 2, 0, 0'

Now you can see that even if a column has internal comma, such as the `host_verifications`, they will not be a problem because we give a quotation mark for a string so the SQL will understand that the comma will not be read as a command to fill the next column.

Finally, We need to transform a missing value `nan` into explicit NULL since SQL only consisder NULL as the legitimate missing value.

In [25]:
# Replace missing value with explicit NULL
value_data = re.sub('\\bnan\\b', 'NULL', value_data)

value_data

'120437, "https://www.airbnb.com/users/show/120437", "Nuttee", "2010-05-08", "Bangkok", "Hi All, I am nuttee patranavik from Bangkok, Thailand.\r\nalways travel but easy to connect via airbnb..", "within a few hours", "100%", NULL, False, "https://a0.muscache.com/im/pictures/user/d05a6b51-b459-4eb8-a01e-8c55122b0132.jpg?aki_policy=profile_small", "https://a0.muscache.com/im/pictures/user/d05a6b51-b459-4eb8-a01e-8c55122b0132.jpg?aki_policy=profile_x_medium", "Victory Monument", 2.0, 2.0, "email, phone, reviews, jumio, offline_government_id, selfie, government_id, identity_manual", True, True, 2, 2, 0, 0'

The full query for the first row is as follow. This is the query or command to store the first observation into MySQL.

In [26]:
"INSERT INTO host_info (" + column_name + ") VALUES (" + value_data + ")"

'INSERT INTO host_info (host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms) VALUES (120437, "https://www.airbnb.com/users/show/120437", "Nuttee", "2010-05-08", "Bangkok", "Hi All, I am nuttee patranavik from Bangkok, Thailand.\r\nalways travel but easy to connect via airbnb..", "within a few hours", "100%", NULL, False, "https://a0.muscache.com/im/pictures/user/d05a6b51-b459-4eb8-a01e-8c55122b0132.jpg?aki_policy=profile_small", "https://a0.muscache.com/im/pictures/user/d05a6b51-b459-4eb8-a01e-8c55122b0132.jpg?aki_policy=profile_x_medium", "Victory Monument", 2.0, 2.0, "email, 

Now that you have understand how to create a query for inserting a single row of data, we will automate this process by creating a function. The following is the full code to insert multiple row of data simultaneously into MySQL. You just need to enter the initial pandas dataframe and the target table in the MySQL server.

To insert multiple row to data, the SQL command template is as follows:

> INSERT INTO table_name (column_1, column_2, ...)\
VALUES\
(value_row_1_column_1, value_row_1_column_2, ...),\
(value_row_2_column_1, value_row_2_column_2, ...),\
(value_row_3_column_1, value_row_3_column_2, ...)

In [27]:
def insert_to_sql(data, table_name):
    
    # Start time counter
    start = time.time()
    
    # Convert column name to single string
    column_name = data.columns.to_list()
    column_name = ",".join(x for x in column_name)
    
    # Create initial query
    query = "INSERT INTO " + table_name + " (" + column_name + ") VALUES" 
    
    # Preparing data
    
    print("Preparing data")
    
    # Create empty list to store input for each row
    value_list = []
    
    for i in range(data.shape[0]):
        
        # Join all values in a single row into a single string
        value_data = ", ".join(str(x) for x in data.iloc[i])
    
        # Add bracket at start and end of a single row insertion
        value_data = "(" + value_data + ")"
    
        # Add the string into list of input
        value_list.append(value_data)

    # Join all string into a single giant string
    join_value = ", ".join( x for x in value_list)

    # Join initial query with the data value
    query = " ".join( [query, join_value])

    # Replace missing value with explicit NULL
    query = re.sub("\\bnan\\b", "NULL", query)   
    query = re.sub("\\\\", "", query)   
    
    print("Inserting data to MySQL")

    # Execute query to MySQL
    cursor = mydb.cursor()
    cursor.execute(query)
    mydb.commit()
    cursor.close()
    
    # Calculate run time 
    end = time.time()
    print('Processing time: %.4f seconds' %(end - start) )

Let's start inserting host data into MySQL `host_info` table.

```
insert_to_sql(df_host, "host_info")
```

If, for some reason you want to delete all the records in your MySQL table, you can run the following SQL query:

> DELETE FROM table_name

```
query = """DELETE FROM host_info""" 

cursor = mydb.cursor() 
cursor.execute(query) 
mydb.commit() 
cursor.close()
```

### Query the Host Table

Now we will try to do some query to the `host_info` table, starting with connect to the MySQL.

Let's try to get the first 10 observation from the `host_info` table. The **LIMIT** command is the same as **head()** in python and will give us the first n row.

In [28]:
pd.read_sql("SELECT * FROM host_info LIMIT 10", mydb)

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,31417,https://www.airbnb.com/users/show/31417,Kami,2009-08-13,"Bangkok, Bangkok, Thailand",One of the first users of Airbnb in 2008. Freq...,None,None,None,0,...,None,0,0,"email, phone, reviews, jumio, offline_governme...",1,1,1,1,0,0
1,58920,https://www.airbnb.com/users/show/58920,Gael,2009-12-01,"Ko Samui, Surat Thani, Thailand","I'm french, living in Samui and Bangkok.\n\nI ...",within a few hours,100%,100%,0,...,None,5,5,"email, phone, facebook, reviews, offline_gover...",1,1,2,2,0,0
2,70413,https://www.airbnb.com/users/show/70413,Patiyuth,2010-01-13,"Bangkok, Thailand","P&R Residence, Bangkok\nSituated in the center...",None,None,None,0,...,Silom,0,0,"email, phone, facebook, reviews",1,0,3,0,3,0
3,108793,https://www.airbnb.com/users/show/108793,Eddie,2010-04-15,bangkok,I have been living in London for 3 years and r...,within an hour,100%,100%,1,...,None,6,6,"email, phone, google, reviews, manual_offline,...",1,1,4,4,0,0
4,120437,https://www.airbnb.com/users/show/120437,Nuttee,2010-05-08,Bangkok,"Hi All, I am nuttee patranavik from Bangkok, T...",within a few hours,100%,None,0,...,Victory Monument,2,2,"email, phone, reviews, jumio, offline_governme...",1,1,2,2,0,0
5,120462,https://www.airbnb.com/users/show/120462,Donald,2010-05-08,Thailand,None,None,None,None,0,...,None,1,1,"email, phone",1,0,1,0,1,0
6,120541,https://www.airbnb.com/users/show/120541,Emy,2010-05-08,"Bangkok, Thailand",None,None,None,None,0,...,None,2,2,"email, phone",1,0,2,1,1,0
7,121622,https://www.airbnb.com/users/show/121622,Anchalee,2010-05-10,"Bangkok, Thailand",Hi my name is Anchalee Topeongpong from Bangko...,None,None,None,0,...,None,2,2,"email, phone, facebook, reviews",1,0,1,1,0,0
8,121848,https://www.airbnb.com/users/show/121848,Jaruwan,2010-05-10,"Bangkok, Thailand",Hi I'm Jaruwan. I love to prepare everything o...,within a day,100%,None,0,...,Chatuchak,1,1,"email, phone, facebook, reviews, jumio, selfie...",1,1,1,1,0,0
9,123784,https://www.airbnb.com/users/show/123784,Familyroom,2010-05-12,Bangkok Thailand,Welcome to Familyroom Apartment.\n\nA Chic and...,None,None,None,0,...,Bang Kapi,1,1,"email, phone",1,0,1,0,1,0


You can try to do conditional filtering of the data. For example, let's try to get the first 5 host who is considered as a superhost. To create a condition, you can use **WHERE** command.

In [29]:
query = """
SELECT * FROM host_info 
WHERE host_is_superhost = TRUE 
LIMIT 5
"""

pd.read_sql(query, mydb)

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,108793,https://www.airbnb.com/users/show/108793,Eddie,2010-04-15,bangkok,I have been living in London for 3 years and r...,within an hour,100%,100%,1,...,None,6,6,"email, phone, google, reviews, manual_offline,...",1,1,4,4,0,0
1,153793,https://www.airbnb.com/users/show/153793,Pemika,2010-06-28,"Bangkok, Bangkok, Thailand",Hi!! I'm a Real Estate Agent with a Tourism ba...,within an hour,100%,100%,1,...,Phra Khanong,9,9,"email, phone, google, jumio, offline_governmen...",1,1,7,7,0,0
2,210574,https://www.airbnb.com/users/show/210574,Norman,2010-08-23,"Doha, Doha, Qatar",I travelled to over 55 countries for business ...,within an hour,100%,67%,1,...,None,4,4,"email, phone, facebook, reviews, jumio, selfie...",1,1,2,1,1,0
3,292635,https://www.airbnb.com/users/show/292635,Sukanya,2010-11-20,Bangkok,"Hey people , I am Sukanya , I live in Bangkok ...",within a few hours,100%,100%,1,...,Dusit,6,6,"email, phone, facebook, reviews, jumio, offlin...",1,1,6,2,4,0
4,298572,https://www.airbnb.com/users/show/298572,Piangporn,2010-11-25,"Tokyo, Japan",I love travel a lot and I understand what trav...,within a few hours,100%,100%,1,...,None,1,1,"email, phone, facebook, reviews, jumio, govern...",1,0,1,1,0,0


You can get the number of row in the data using `COUNT( * )`

In [30]:
query = """
SELECT COUNT(*) as frequency FROM host_info
"""

pd.read_sql(query, mydb)

,frequency
0,8296


## Listing
### Processing Listing Table

We will do full analysis of the data and understand different kind of SQL command in another post. For now, we will finish inserting all data into the database. Let's continue processing the remaining column from the listing dataset.

In [31]:
# Remove column related to host except host_id
df_new_listing = df_listing.drop( df_host.columns[1:], axis = 1).copy()

df_new_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19289 entries, 0 to 19288
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            19289 non-null  int64  
 1   listing_url                   19289 non-null  object 
 2   scrape_id                     19289 non-null  int64  
 3   last_scraped                  19289 non-null  object 
 4   name                          19280 non-null  object 
 5   description                   18118 non-null  object 
 6   neighborhood_overview         11099 non-null  object 
 7   picture_url                   19289 non-null  object 
 8   host_id                       19289 non-null  int64  
 9   neighbourhood                 11099 non-null  object 
 10  neighbourhood_cleansed        19289 non-null  object 
 11  neighbourhood_group_cleansed  0 non-null      float64
 12  latitude                      19289 non-null  float64
 13  l

Some columns, such as the `calendar_updated`, `license`, and `bathrooms` are only consists of missing values. The `scrape_id` is also has no apparent meaning for now since we already has the `last_scraped` column to show us the lates date of scraping. These columns are not informative, so we will drop them.

In [32]:
# Delete columns with all missing values and scrape_id
df_new_listing.drop(['license', 'calendar_updated', 'bathrooms', 'scrape_id'], axis = 1, inplace = True)

Next, we will convert columns that should has a boolean/logical value, including `has_availability` and `instant_bookable`.

In [33]:
for col in ['has_availability', 'instant_bookable']:
    
    df_new_listing[col] = list(map(lambda x: True if x == "t" else False if x == "f" else np.nan, df_new_listing[col]))

We will continue by transforming the `price` column into decimal/numeric values by removing the dollar sign from the string.

In [34]:
df_new_listing['price'] = list(map(lambda x: float(re.sub('[$,]', '', x)), df_new_listing['price']))

df_new_listing['price'][0:5]

0    1694.0
1     924.0
2    1188.0
3    1095.0
4     800.0
Name: price, dtype: float64

Now we will continue transforming the `amenities` with the same treatment as the `host_verification` column from the host dataset.

In [35]:
df_new_listing['amenities'] = list(map(lambda x: re.sub("[\"\'\[\]]", '', x), df_new_listing['amenities']))

df_new_listing['amenities'][0:5]

0    Microwave, Pool, Smoke alarm, Dryer, Long term...
1    Cable TV, Kitchen, Air conditioning, Free park...
2    Cable TV, TV, Pool, Shampoo, Hot tub, Heating,...
3    Dedicated workspace, Air conditioning, Hair dr...
4                              Long term stays allowed
Name: amenities, dtype: object

We will clean the string object with the `clean_char()` function from the previous step.

In [36]:
df_new_listing = clean_char(df_new_listing)

### Create Listing Table

Now we will start creating a listing table in the MySQL server. First, let's check the length of each string column in the data.

In [37]:
check_char(df_new_listing.select_dtypes('object'))

Maximum Character Length
listing_url: 39
last_scraped: 12
name: 257
description: 1002
neighborhood_overview: 1002
picture_url: 128
neighbourhood: 87
neighbourhood_cleansed: 22
property_type: 36
room_type: 17
bathrooms_text: 19
amenities: 1081
calendar_last_scraped: 12
first_review: 12
last_review: 12


Let's start writing the query for creating the table. For a numeric values such as price, we will use **Decimal** data type while the rest is the same as the host table. The primary key for the listing table is the `id`, which indicate the listing id. The foreign key for the listing table is `host_id`, which we can use to join the listing table with the host_info table for analysis.

In [38]:
query =  """
CREATE TABLE listing (
id INT,
listing_url VARCHAR(100),
last_scraped DATE,
name VARCHAR(500),
description VARCHAR(2000),
neighborhood_overview VARCHAR(2000),
picture_url VARCHAR(500),
host_id INT,
neighbourhood VARCHAR(100),
neighbourhood_cleansed VARCHAR(100),
neighbourhood_group_cleansed VARCHAR(100),
latitude DECIMAL(25,18),
longitude DECIMAL(25, 18),
property_type VARCHAR(100),
room_type VARCHAR(100),
accommodates INT,
bathrooms_text VARCHAR(100),
bedrooms INT,
beds INT,
amenities VARCHAR(2000),
price DECIMAL(15, 5),
minimum_nights INT,
maximum_nights INT,
minimum_minimum_nights INT,
maximum_minimum_nights INT,
minimum_maximum_nights INT,
maximum_maximum_nights INT,
minimum_nights_avg_ntm DECIMAL(16, 5),
maximum_nights_avg_ntm DECIMAL(16, 5),
has_availability BOOLEAN,
availability_30 INT,
availability_60 INT,
availability_90 INT,
availability_365 INT,
calendar_last_scraped DATE,
number_of_reviews INT,
number_of_reviews_ltm INT,
number_of_reviews_l30d INT,
first_review DATE,
last_review DATE,
review_scores_rating DECIMAL(10, 5),
review_scores_accuracy DECIMAL(10, 5),
review_scores_cleanliness DECIMAL(10, 5),
review_scores_checkin DECIMAL(10, 5),
review_scores_communication DECIMAL(10, 5),
review_scores_location DECIMAL(10, 5),
review_scores_value DECIMAL(10, 5),
instant_bookable BOOLEAN,
reviews_per_month DECIMAL(10, 5),
PRIMARY KEY(id),
FOREIGN KEY(host_id) REFERENCES host_info(host_id)
)
"""

query

'\nCREATE TABLE listing (\nid INT,\nlisting_url VARCHAR(100),\nlast_scraped DATE,\nname VARCHAR(500),\ndescription VARCHAR(2000),\nneighborhood_overview VARCHAR(2000),\npicture_url VARCHAR(500),\nhost_id INT,\nneighbourhood VARCHAR(100),\nneighbourhood_cleansed VARCHAR(100),\nneighbourhood_group_cleansed VARCHAR(100),\nlatitude DECIMAL(25,18),\nlongitude DECIMAL(25, 18),\nproperty_type VARCHAR(100),\nroom_type VARCHAR(100),\naccommodates INT,\nbathrooms_text VARCHAR(100),\nbedrooms INT,\nbeds INT,\namenities VARCHAR(2000),\nprice DECIMAL(15, 5),\nminimum_nights INT,\nmaximum_nights INT,\nminimum_minimum_nights INT,\nmaximum_minimum_nights INT,\nminimum_maximum_nights INT,\nmaximum_maximum_nights INT,\nminimum_nights_avg_ntm DECIMAL(16, 5),\nmaximum_nights_avg_ntm DECIMAL(16, 5),\nhas_availability BOOLEAN,\navailability_30 INT,\navailability_60 INT,\navailability_90 INT,\navailability_365 INT,\ncalendar_last_scraped DATE,\nnumber_of_reviews INT,\nnumber_of_reviews_ltm INT,\nnumber_of_re


Let's run the query.

```
cursor = mydb.cursor() 

# Create listing table
cursor.execute(query) 
mydb.commit() 
cursor.close()
```

You can check the listing table with the following query.

In [39]:
pd.read_sql("DESCRIBE listing", mydb)

,Field,Type,Null,Key,Default,Extra
0,id,b'int',NO,PRI,None,
1,listing_url,b'varchar(100)',YES,,None,
2,last_scraped,b'date',YES,,None,
3,name,b'varchar(500)',YES,,None,
4,description,b'varchar(2000)',YES,,None,
5,neighborhood_overview,b'varchar(2000)',YES,,None,
6,picture_url,b'varchar(500)',YES,,None,
7,host_id,b'int',YES,MUL,None,
8,neighbourhood,b'varchar(100)',YES,,None,
9,neighbourhood_cleansed,b'varchar(100)',YES,,None,


### Insert Data to Listing Table

Now we will insert each row into the table using the same function that we have defined earlier. Unfortunately, MySQL has maximum allowed packet that goes in a single query. You can check with the following query.

In [40]:
pd.read_sql("SHOW VARIABLES LIKE 'max_allowed_packet'", mydb)

,Variable_name,Value
0,max_allowed_packet,25165824


Since our data is larger than the allowed packet, we need to split our data into smaller dataset. Here, we will split the data into 10 equal parts.

Read more about this problem at [here](https://dev.mysql.com/doc/refman/5.7/en/packet-too-large.html)

In [41]:
data_split = np.linspace(0, df_new_listing.shape[0], 10).astype('int')

data_split[0:10]

array([    0,  2143,  4286,  6429,  8572, 10716, 12859, 15002, 17145,
       19289])

The first batch will contain data from the first row (0) to the 2143rd row.

In [42]:
df_new_listing.iloc[ data_split[0]:data_split[(0+1)] ]

,id,listing_url,last_scraped,name,description,neighborhood_overview,picture_url,host_id,neighbourhood,neighbourhood_cleansed,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,27934,"""https://www.airbnb.com/rooms/27934""","""2021-03-22""","""Nice room with superb city view""","""Our cool and comfortable one bedroom apartmen...","""It is very center of Bangkok and easy access ...","""https://a0.muscache.com/pictures/566374/23157...",120437,"""Samsen Nai, Bangkok, Thailand""","""Ratchathewi""",...,"""2020-01-06""",97.0,10.0,10.0,10.0,10.0,9.0,9.0,False,0.60
1,27942,"""https://www.airbnb.com/rooms/27942""","""2021-03-22""","""Town House (1) bedroom available""","""<b>The space</b><br />Townhouse with 2 bedroo...",NaN,"""https://a0.muscache.com/pictures/155668/afc77...",120462,NaN,"""Bangkok Yai""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2,27979,"""https://www.airbnb.com/rooms/27979""","""2021-03-31""","""Easy going landlord,easy place""","""<b>The space</b><br />'The village Condominiu...",NaN,"""https://a0.muscache.com/pictures/106247594/1d...",120541,NaN,"""Bang Na""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,28354,"""https://www.airbnb.com/rooms/28354""","""2021-04-01""","""Place near Metro and Shopping Mall ( Monthly ...","""Apartment is near 2 shopping mall Central Pla...",NaN,"""https://a0.muscache.com/pictures/3097966/efa6...",121848,NaN,"""Chatu Chak""",...,"""2019-12-09""",96.0,10.0,10.0,10.0,10.0,9.0,9.0,True,0.40
4,28745,"""https://www.airbnb.com/rooms/28745""","""2021-03-30""","""modern-style apartment in Bangkok""","""A modern-style apartment situated in a cool n...","""Ramkumheang neigbourhood - hip place with foo...","""https://a0.muscache.com/pictures/160369/92a7e...",123784,"""Bangkok, Thailand""","""Bang Kapi""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,10528084,"""https://www.airbnb.com/rooms/10528084""","""2021-03-29""","""2 BR APT in the hip area of Bkk""","""Make your self at home in this two bedrooms a...","""The condo is 2 mins walk to Thonglor area - t...","""https://a0.muscache.com/pictures/ea559265-1ed...",4282827,"""Bangkok, Thailand""","""Vadhana""",...,"""2018-05-08""",97.0,10.0,10.0,10.0,10.0,9.0,10.0,False,0.32
2139,10529822,"""https://www.airbnb.com/rooms/10529822""","""2021-03-31""","""Circle condominium""","""Nice 1 bed & living room<br /><br /><b>The sp...",NaN,"""https://a0.muscache.com/pictures/c5e77227-83f...",53251672,NaN,"""Ratchathewi""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2140,10540077,"""https://www.airbnb.com/rooms/10540077""","""2021-03-22""","""Cheap clean safe convenience home""","""Cheap clean safe friendly and most convenienc...",NaN,"""https://a0.muscache.com/pictures/b06196aa-2a7...",54199136,NaN,"""Nong Khaem""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2141,10558675,"""https://www.airbnb.com/rooms/10558675""","""2021-04-01""","""ModernStay-2BRCityView-NanaBTS-80sqm-FreeNetf...","""Right at the heart of nana area, the newly re...","""Lots of bars, restaurants and massage parlors...","""https://a0.muscache.com/pictures/4d7b84c8-532...",54458768,"""Bangkok, Krung Thep Maha Nakhon, Thailand""","""Khlong Toei""",...,"""2020-07-01""",95.0,10.0,9.0,10.0,10.0,10.0,9.0,True,2.67


We will insert the data one batch at a time with loop.

```
# Insert data per batch
for i in range(len(data_split)):
    
    if i < (len(data_split) -1) :
        
        print("Uploading rows: " + str( data_split[i] ) + '-' + str( data_split[ (i+1) ] ))
        
        # Select data according to batch
        selected_data = df_new_listing.iloc[ data_split[i]:data_split[(i+1)] ]
        
        # Insert data to MySQL
        insert_to_sql(selected_data, 'listing2')
        
        print('-----------------')
```

### Query the Listing Table

Let's do some simple query to check the data that we have inserted.

Let's check how many rows are there on the listing table.

In [43]:
pd.read_sql("SELECT COUNT(*) as frequency FROM listing", mydb)

,frequency
0,19289


I want to see the top 10 property type based on the frequency from the listing. We can use **COUNT** to get the number of row and use **GROUP BY** to make sure we count the number of row for each property type. To sort the value, we can use **ORDER BY**, folllowed by **DESC** to indicate that we want to sort descending and finally we only take the first 10 row with **LIMIT**.

In [44]:
query = """
SELECT COUNT(*) as frequency, property_type FROM listing GROUP BY property_type ORDER BY COUNT(*) DESC LIMIT 10
"""

pd.read_sql(query, mydb)

,frequency,property_type
0,5102,Entire apartment
1,3679,Entire condominium
2,2057,Private room in apartment
3,971,Private room in condominium
4,767,Private room in house
5,692,Room in boutique hotel
6,574,Room in hotel
7,573,Private room in hostel
8,573,Private room in townhouse
9,491,Entire house


How about the top 10 most expensive listing? We can check the sorting the data with the `price` column.


In [45]:
query = """
SELECT price, property_type, neighbourhood FROM listing ORDER BY price DESC LIMIT 10
"""

pd.read_sql(query, mydb)

,price,property_type,neighbourhood
0,312839.0,Private room in apartment,"Bangkok, Thailand"
1,300000.0,Private room in house,None
2,300000.0,Entire house,"Bangkok, Krung Thep Maha Nakhon, Thailand"
3,236551.0,Entire apartment,"Khet Khlong Toei, Krung Thep Maha Nakhon, Thai..."
4,186789.0,Entire villa,None
5,160000.0,Entire apartment,"Bangkok, Krung Thep Maha Nakhon, Thailand"
6,140774.0,Private room in apartment,"Khet Khlong Toei, Krung Thep Maha Nakhon, Thai..."
7,100000.0,Shared room in house,"Bangkok, Thailand"
8,100000.0,Shared room in house,"Bangkok, Thailand"
9,100000.0,Private room in house,Thailand


# Conclusion

If you have done with doing some query with your MySQL server, don't forget to close the connection to the database.

In [46]:
mydb.close()

I hope that the notebook has fulfilled the following goal that we have stated earlier:

- Create a database with MySQL server
- Create multiple table
- Insert data into SQL table 
- Write query to do simple data collection from MySQL database

For my next post, I will write on how to do some data analysis using SQL Query.